# 第３回講義 宿題

## 課題. MNISTをk-NNで学習せよ

### 注意
- homework関数を完成させて提出してください
    - 訓練データはtrain_X, train_y, テストデータはtest_Xで与えられます
    - train_Xとtrain_yをtrain_X, train_yとvalid_X, valid_yに分けるなどしてモデルを学習させてください
    - test_Xに対して予想ラベルpred_yを作り, homework関数の戻り値としてください\
- pred_yのtest_yに対する精度(F値)で評価します
- 全体の実行時間がiLect上で60分を超えないようにしてください
- homework関数の外には何も書かないでください (必要なものは全てhomework関数に入れてください)

- k-NNの実装にscikit-learnなどのライブラリを使わないでください

次のセルのhomework関数を完成させて提出してください

In [37]:
def homework(train_X, train_y, test_X):
    # WRITE ME
    # hyperparameter 
    k = 10
    
    
    # normalization
    norm_train_x = np.linalg.norm(train_X, axis=1)
    norm_test_x = np.linalg.norm(test_X, axis=1)
    train_X = train_X / norm_train_x.reshape(len(norm_train_x), 1)
    test_X = test_X / norm_test_x.reshape(len(norm_test_x), 1)
    
    pred_y = []
    for i in range(test_X.shape[0]):
        scores = np.dot(train_X, test_X[i])
        # sort by score 
        ranking = sorted([ [s, l] for s, l  in zip(scores, train_y)], reverse = True)
        most_similar = np.array(ranking[:k])
        x= np.array(np.unique(most_similar, return_counts=True))
        pred = x[0][np.argmax(x[1])]
        pred_y.append(pred)
    
    return np.array(pred_y)

- 以下のvalidate_homework関数を用いてエラーが起きないか動作確認をして下さい。
- 評価は以下のscore_homework関数で行われますが、random_stateの値は変更されます。

In [2]:
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split

import numpy as np

def load_mnist():
    mnist = fetch_mldata('MNIST original')
    mnist_X, mnist_y = shuffle(mnist.data.astype('float32'),
                               mnist.target.astype('int32'), random_state=42)

    mnist_X = mnist_X / 255.0

    return train_test_split(mnist_X, mnist_y,
                test_size=0.2,
                random_state=42)

def validate_homework():
    train_X, test_X, train_y, test_y = load_mnist()

    # validate for small dataset
    train_X_mini = train_X[:1000]
    train_y_mini = train_y[:1000]
    test_X_mini = test_X[:1000]
    test_y_mini = test_y[:1000]

    pred_y = homework(train_X_mini, train_y_mini, test_X_mini)
    print(f1_score(test_y_mini, pred_y, average='macro'))

def score_homework():
    train_X, test_X, train_y, test_y = load_mnist()
    pred_y = homework(train_X, train_y, test_X)
    print(f1_score(test_y, pred_y, average='macro'))

In [38]:
validate_homework()
#score_homework()

0.865915263722


In [42]:
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
params = {
    "n_neighbors": [2*i for i in range(2, 15)],
    "weights" :['uniform', "distance"],
    "algorithm":['auto', 'ball_tree', 'kd_tree', 'brute'],
    # "leaf_size": [10,20,30,40],
}
          
clf = model_selection.GridSearchCV(knn, params, scoring="f1", n_jobs=-1, cv=4)

In [44]:
train_X, test_X, train_y, test_y = load_mnist()
clf.fit(train_X, train_y) 

OSError: [Errno 28] No space left on device

In [ ]:
clf.best_scores_

In [ ]:
clf.best_params_